In [15]:
import os,sys
import numpy as np

from keras.datasets import imdb
from keras import backend as K
from sklearn.model_selection import train_test_split
import pandas as pd
from keras.utils.np_utils import to_categorical
from sklearn.preprocessing import Normalizer
from keras.models import Sequential
from keras.layers import Convolution1D, Dense, Dropout, Flatten, MaxPooling1D, Conv2D, Conv1D, Dropout
from keras.utils import np_utils
import numpy as np

from keras import callbacks
from keras.layers import LSTM, GRU, SimpleRNN
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, CSVLogger

from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score, cross_validate
import pandas as pd
from sklearn.model_selection import GridSearchCV
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
from sklearn.metrics import precision_recall_curve
from sklearn.preprocessing import Binarizer
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.neighbors import KNeighborsClassifier
from lightgbm import LGBMClassifier
from sklearn.feature_selection import chi2, SelectKBest
import timeit

df = pd.read_csv('./UNSW_NB15_testing-set.csv')
df_test = pd.read_csv('./UNSW_NB15_training-set.csv')



In [27]:
df.columns

Index(['id', 'dur', 'proto', 'service', 'state', 'spkts', 'dpkts', 'sbytes',
       'dbytes', 'rate', 'sttl', 'dttl', 'sload', 'dload', 'sloss', 'dloss',
       'sinpkt', 'dinpkt', 'sjit', 'djit', 'swin', 'stcpb', 'dtcpb', 'dwin',
       'tcprtt', 'synack', 'ackdat', 'smean', 'dmean', 'trans_depth',
       'response_body_len', 'ct_srv_src', 'ct_state_ttl', 'ct_dst_ltm',
       'ct_src_dport_ltm', 'ct_dst_sport_ltm', 'ct_dst_src_ltm',
       'is_ftp_login', 'ct_ftp_cmd', 'ct_flw_http_mthd', 'ct_src_ltm',
       'ct_srv_dst', 'is_sm_ips_ports', 'attack_cat', 'label'],
      dtype='object')

In [14]:


def label(df, test):
    y_train = df['label']
    y_test = test['label']
    
    X_train = df.drop(['label'],axis = 1)
    X_test = test.drop(['label'],axis=1)
    return y_train, y_test, X_train, X_test


y_train, y_test, X_train, X_test = label(df, df_test)


def label(df):
    
    service = df['service']
    encoder = LabelEncoder() 
    encoder.fit(service)
    services = encoder.transform(service)  
    print('인코딩 변환값: ', services)

    df['service'] = services

    flag = df['state']
    encoder = LabelEncoder() 
    encoder.fit(flag)
    flags = encoder.transform(flag)  
    print('인코딩 변환값: ', flags)

    df['state'] = flags

    protocol_type = df['proto']
    encoder = LabelEncoder() 
    encoder.fit(protocol_type)
    protocol_types = encoder.transform(protocol_type)  
    print('인코딩 변환값: ', protocol_types)

    df['proto'] = protocol_types
    return df
    
total_df = X_train, X_test
    
for i in total_df:
    a = label(i)
    

def accuracy(y_test, pred):
    accuracy = accuracy_score(y_test, pred)
    precision = precision_score(y_test, pred)
    recall = recall_score(y_test, pred)
    f1 = f1_score(y_test, pred)
    #confusion = skplt.metrics.plot_confusion_matrix(y_test, pred)
    a = np.round(accuracy*100,2),np.round(precision*100,2),np.round(recall*100,2),np.round(f1*100,2)
    print('Accuracy \t {0:.2f}% ,\n Precision \t{1:.2f}%,\n Recall \t{2:.2f}%,\n F1_score \t{3:.2f}%'.format(accuracy*100,precision*100,
                                      recall*100, f1*100))
    return a
    
total_colNames=list(X_train)
total_colNames_test=list(y_train)



인코딩 변환값:  [0 0 0 ... 0 0 0]
인코딩 변환값:  [4 4 4 ... 4 4 4]
인코딩 변환값:  [117 117 117 ...   6   6 117]
인코딩 변환값:  [0 0 0 ... 2 2 2]
인코딩 변환값:  [2 2 2 ... 3 3 3]
인코딩 변환값:  [113 113 113 ... 119 119 119]


In [3]:
X_train.drop(['attack_cat'], axis = 1, inplace = True)
X_test.drop(['attack_cat'], axis = 1, inplace = True)

In [4]:
from sklearn.feature_selection import f_classif 

for i in range(43,44):
    selector = SelectKBest(chi2, k=i)
    X_st = selector.fit_transform(X_train, y_train)
    X_st_test = selector.transform(X_test)
    true=selector.get_support()
    total_colNames_index=[z for z, x in enumerate(true) if x]
    skb_total=list( total_colNames[i] for i in total_colNames_index )
    print('SelectKBest의 갯수',len(skb_total))
    
    sc  = StandardScaler()
    new_df = sc.fit_transform(X_st)
    new_df_test = sc.fit_transform(X_st_test)
    
    X_total = pd.DataFrame(new_df, columns = skb_total )
    X_total_test = pd.DataFrame(new_df_test, columns = skb_total)
    dt_clf = DecisionTreeClassifier()
    dt_clf.fit(X_total, y_train)
    total_dt_pred = dt_clf.predict(X_total_test)
    print(np.round(accuracy_score(total_dt_pred, y_test)*100,2))



SelectKBest의 갯수 43
53.28


In [5]:
from sklearn.svm import SVC
import time
    
start = time.time()
svm = SVC(kernel='rbf')
svm.fit(X_total, y_train)
svm_pred =svm.predict(X_total_test)
end = time.time()
total = end - start
print(accuracy(svm_pred, y_test))
print(total)


Accuracy 	 84.70% ,
 Precision 	86.14%,
 Recall 	90.90%,
 F1_score 	88.46%
(84.7, 86.14, 90.9, 88.46)
126.80517053604126


In [6]:
dt_clf = DecisionTreeClassifier( random_state =15)
start = time.time()
dt_clf.fit(X_total, y_train)
dt_pred = dt_clf.predict(X_total_test)
end = time.time()
total = end - start
print(total)
print(accuracy(dt_pred, y_test))



0.6070976257324219
Accuracy 	 41.87% ,
 Precision 	53.63%,
 Recall 	57.87%,
 F1_score 	55.67%
(41.87, 53.63, 57.87, 55.67)


In [7]:
start = time.time()

lgbm_clf = LGBMClassifier(random_state = 1)
lgbm_clf.fit(X_total, y_train)
lgbm_pred = lgbm_clf.predict(X_total_test)
end = time.time()
total = end - start
print(total)
print(accuracy(lgbm_pred, y_test))



0.9002072811126709
Accuracy 	 30.52% ,
 Precision 	37.96%,
 Recall 	48.66%,
 F1_score 	42.65%
(30.52, 37.96, 48.66, 42.65)


In [8]:
knn_clf = KNeighborsClassifier(n_neighbors =6)
start = time.time()

knn_clf.fit(X_total, y_train)
knn_pred = knn_clf.predict(X_total_test)
end = time.time()
total = end - start
print(total)
accuracy(knn_pred, y_test)








211.8557288646698
Accuracy 	 79.77% ,
 Precision 	76.36%,
 Recall 	92.61%,
 F1_score 	83.71%


(79.77, 76.36, 92.61, 83.71)

In [9]:
start = time.time()

rf_clf = RandomForestClassifier(max_depth= 7,random_state= 1)
rf_clf.fit(X_total, y_train)
rf_pred = rf_clf.predict(X_total_test)
end = time.time()
total = end - start
print(total)
print(accuracy(rf_pred, y_test))

from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import BernoulliNB
start = time.time()

6.258786916732788
Accuracy 	 73.03% ,
 Precision 	67.88%,
 Recall 	90.05%,
 F1_score 	77.41%
(73.03, 67.88, 90.05, 77.41)


In [10]:
start = time.time()

nb_clf = BernoulliNB()
nb_clf.fit(X_total, y_train)
nb_pred = nb_clf.predict(X_total_test)
end = time.time()
total = end - start
print(total)
print(accuracy(nb_pred, y_test))

0.1825098991394043
Accuracy 	 73.61% ,
 Precision 	65.45%,
 Recall 	93.94%,
 F1_score 	77.15%
(73.61, 65.45, 93.94, 77.15)


In [11]:
from sklearn.ensemble import AdaBoostClassifier
ada_clf = AdaBoostClassifier(random_state = 1)
start = time.time()

ada_clf.fit(X_total, y_train)
ada_pred = ada_clf.predict(X_total_test)
end = time.time()
total = end - start
print(total)
print(accuracy(ada_pred, y_test))


8.367164850234985
Accuracy 	 28.39% ,
 Precision 	37.84%,
 Recall 	46.78%,
 F1_score 	41.84%
(28.39, 37.84, 46.78, 41.84)


In [28]:
start = time.time()

lr_clf = LogisticRegression(max_iter = 1000)
lr_clf.fit(X_total, y_train)
lr_pred = lr_clf.predict(X_total_test)
end = time.time()
total = end - start
print(total)
print(accuracy(lr_pred, y_test))

1.6216328144073486
Accuracy 	 85.40% ,
 Precision 	87.55%,
 Recall 	90.68%,
 F1_score 	89.09%
(85.4, 87.55, 90.68, 89.09)


In [42]:
final_clf =  LGBMClassifier(random_state = 1)

In [59]:
def stacking(clf1,clf2,clf3,clf4,clf5,clf6, clf8, clf9 ,final_clf,x_train,x_test,y_train,y_test):

    
    clf1 = SVC(kernel='rbf')
    clf1 .fit(X_total, y_train)
    svm_pred =clf1 .predict(X_total_test)

    clf2 = DecisionTreeClassifier(max_depth= 5)
    clf2.fit(X_total, y_train)
    dt_pred = clf2.predict(X_total_test)

    clf3 = LGBMClassifier(max_depth = 7, num_leaves = 36)
    clf3.fit(X_total, y_train)
    lgbm_pred = clf3.predict(X_total_test)

    clf4 = KNeighborsClassifier(n_neighbors =5)
    clf4.fit(X_total, y_train)
    knn_pred = clf4.predict(X_total_test)

  
    clf5 = RandomForestClassifier(max_depth= 7,random_state= 1)
    clf5.fit(X_total, y_train)
    rf_pred = clf5.predict(X_total_test)


    clf6 = BernoulliNB()
    clf6.fit(X_total, y_train)
    nb_pred = clf6.predict(X_total_test)
    def random_nomal_initializer_with_stddev_1():
        return tf.keras.initializers.RandomNormal(mean = 0.0, stddev=1.0, seed=None)



    def ann_model():
        ann = Sequential()
        ann.add(Dense(41,
                    activation = 'relu',
                    kernel_initializer =random_nomal_initializer_with_stddev_1(),
                    bias_initializer = random_nomal_initializer_with_stddev_1()))

        ann.add(Dense(20, activation = 'relu',
                     kernel_initializer =random_nomal_initializer_with_stddev_1(),
                     bias_initializer = random_nomal_initializer_with_stddev_1()))
        ann.add(Dense(10, activation = 'relu',
                     kernel_initializer =random_nomal_initializer_with_stddev_1(),
                     bias_initializer = random_nomal_initializer_with_stddev_1()))
        ann.add(Dense(5, activation = 'relu',
                     kernel_initializer =random_nomal_initializer_with_stddev_1(),
                     bias_initializer = random_nomal_initializer_with_stddev_1()))


        ann.add(Dense(1,activation = 'sigmoid'))
        ann.compile(loss = 'binary_crossentropy', optimizer = 'Nadam', metrics = ['accuracy'])
        return ann

    
    

    clf8 = AdaBoostClassifier(random_state = 1,n_estimators=100)
    clf8.fit(X_total, y_train)
    ada_pred = clf8.predict(X_total_test)

    clf9 = LogisticRegression(max_iter = 1000)
    clf9.fit(X_total, y_train)
    lr_pred = clf9.predict(X_total_test)



    
    
    preds = np.array([svm_pred, dt_pred, lgbm_pred,knn_pred,rf_pred, nb_pred,ada_pred, lr_pred])
    preds= np.transpose(preds) # or pred.T
    final_clf =  LGBMClassifier(random_state = 1)
    final_clf.fit(preds, y_test)
    finalr_pred = final_clf.predict(preds)
    print(preds.shape, y_test.shape)
    print('Stacking 모델의 예측 정확도 : ')
    b= pd.DataFrame(accuracy(finalr_pred, y_test))
    accuracy(finalr_pred, y_test)
    return b

In [40]:
import time 
from sklearn.feature_selection import f_classif 
b = pd.DataFrame(columns=range(43,44))
c = []
for i in range(43,44):
    start = time.time()

    selector = SelectKBest(f_classif , k=i)
    X_st = selector.fit_transform(X_train, y_train)
    X_st_test = selector.transform(X_test)
    true=selector.get_support()
    total_colNames_index=[z for z, x in enumerate(true) if x]
    skb_total=list( total_colNames[i] for i in total_colNames_index )
    print(skb_total)
    print('SelectKBest의 갯수',len(skb_total))
    
    sc  = StandardScaler()
    new_df = sc.fit_transform(X_st)
    new_df_test = sc.fit_transform(X_st_test)
    
    X_total = pd.DataFrame(new_df, columns = skb_total )
    X_total_test = pd.DataFrame(new_df_test, columns = skb_total)
    print('feature 개수 : ',len(skb_total))
    a = stacking(svm,  dt_clf, lgbm_clf, knn_clf,rf_pred,nb_pred,ada_clf,lr_clf, final_clf,X_total,X_total_test
             ,y_train,y_test)
    a = a[0]
    print('----------------')
    b[i] = a
    end = time.time()
    total_time = end - start
    c.append(total_time)



['id', 'dur', 'proto', 'service', 'state', 'spkts', 'dpkts', 'sbytes', 'dbytes', 'rate', 'sttl', 'dttl', 'sload', 'dload', 'sloss', 'dloss', 'sinpkt', 'dinpkt', 'sjit', 'djit', 'swin', 'stcpb', 'dtcpb', 'dwin', 'tcprtt', 'synack', 'ackdat', 'smean', 'dmean', 'trans_depth', 'response_body_len', 'ct_srv_src', 'ct_state_ttl', 'ct_dst_ltm', 'ct_src_dport_ltm', 'ct_dst_sport_ltm', 'ct_dst_src_ltm', 'is_ftp_login', 'ct_ftp_cmd', 'ct_flw_http_mthd', 'ct_src_ltm', 'ct_srv_dst', 'is_sm_ips_ports']
SelectKBest의 갯수 43
feature 개수 :  43


NameError: name 'stacking' is not defined

In [37]:
X_train.drop('attack_cat', axis =1 , inplace = True)

In [39]:
X_test.drop('attack_cat', axis =1 , inplace = True)

In [63]:
c

[453.2051692008972]

In [60]:
b

,43
0,91.64
1,97.61
2,90.79
3,94.08


In [58]:
c

[454.22031593322754]

In [31]:
preds = np.array([ dt_pred,
                   lgbm_pred, lr_pred,  ada_pred, knn_pred,svm])
print(preds.shape)
preds= np.transpose(preds) # or pred.T
print(preds.shape)
final_clf =  LGBMClassifier(random_state = 1)

final_clf.fit(preds, y_test)
finalr_pred = final_clf.predict(preds)
print('Stacking 모델의 예측 정확도 : ')
accuracy(finalr_pred, y_test)

(6, 175341)
(175341, 6)
Stacking 모델의 예측 정확도 : 
Accuracy 	 93.03% ,
 Precision 	99.32%,
 Recall 	91.22%,
 F1_score 	95.10%


(93.03, 99.32, 91.22, 95.1)